In [ ]:
import uproot as ur
import h5py, sys, os, yaml

import torch, os, yaml, argparse, h5py
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from glob import glob
import uproot as ur
from typing import Callable, cast

import FlavorTag.utils.helperfunctions_ntags as helperfunctions
import FlavorTag.utils.helperfunctions_ParticleNet as helperfunctions_ParticleNet
from FlavorTag.training_data.ParticleNet.create_dataframes_ParticleNet import create_dataframes
from FlavorTag.utils.helperfunctions_ntags import calc_df_specification, get_rename_dicts, \
    extract_feature_columns, ttree_to_array, sanitize_array, init_df

/cvmfs/sw.hsf.org/key4hep/releases/2024-10-03/x86_64-almalinux9-gcc14.2.0-opt/py-tqdm/4.66.3-gay4aw/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
config = '/afs/desy.de/group/flc/pool/bliewert/MarlinWorkdirs/ZHH/dependencies/FlavourTaggingML/FlavorTag/configs/ParticleNet_4tags_v2.yaml'
with open(config) as fs:
    parameters = yaml.safe_load(fs)

In [3]:
mask_b = '/nfs/dust/ilc/user/bliewert/FT_ParticleNet4Tags/FeatureExtraction/output/*_bbbbbb_*.root'
files_b = glob(mask_b)
files_b.sort()

In [4]:
if False:
    files_b[0] = '/afs/desy.de/group/flc/pool/bliewert/MarlinWorkdirs/ZHH/scripts/AIDAFile_.root.root'
    rb = ur.open(files_b[0])

In [7]:
rb['ChargedJetConstituents'].keys()

['Charge',
 'Momentum',
 'MomentumX',
 'MomentumY',
 'MomentumZ',
 'TransverseMomentum',
 'TransverseMomentumLOG',
 'TransverseMomentumFrac',
 'TransverseMomentumFracLOG',
 'Energy',
 'EnergyLOG',
 'TrackMomentumFrac',
 'EnergyFrac',
 'EnergyFracLOG',
 'TrackThetaRelJet',
 'TrackPtRelJet',
 'TrackJetDotProduct',
 'TrackJetDotProductNorm',
 'DeltaRJet',
 'DeltaRJetRapidity',
 'DeltaRTrackSecVertex',
 'PhiRelJet',
 'RapidityRelJet',
 'Rapidity',
 'IsElectron',
 'IsMuon',
 'IsPhoton',
 'IsChargedHadron',
 'IsNeutralHadron',
 'D0Sig',
 'D0',
 'Z0',
 'Z0Sig',
 '3DImpactPar',
 '3DImpactParSig',
 'Chi2NDF',
 'NDF',
 'TrackUsedinPrimVertex',
 'LeptonMomRelJet',
 'LeptonPtRelJet',
 'LeptonMomFraction',
 'CPID11ness',
 'CPID13ness',
 'CPID211ness',
 'CPID321ness',
 'CPID2212ness',
 'D0bprob',
 'D0cprob',
 'D0qprob',
 'Z0bprob',
 'Z0cprob',
 'Z0qprob',
 'JetIndex',
 'Kaoness',
 'WeightedKaonnessMomFrac',
 'isOverlay']

In [8]:
rb['Jets'].num_entries

54

In [9]:
rb['Jets'].keys(filter_name='*Jet*')

['JetEnergy',
 'JetPt',
 'JetMomentum',
 'JetTheta',
 'JetPhi',
 'JetRapidity',
 'JetIndex']

In [10]:
np.array([1,2,3,4,5,6])[[0,3,2,3]]

array([1, 4, 3, 4])

In [155]:
pf_charged_columns, pf_neutral_columns, sv_columns, mappings_by_group = extract_input_columns(
    parameters['inputs'],
    rb['ChargedJetConstituents'].keys(),
    rb['NeutralJetConstituents'].keys(),
    rb['SecondaryVertices'].keys(),
    verbose_fn=print
)

Feature <HCalFrac> not found for charged PFOs for group <HCalFrac> in inputs. Setting to None
Feature <D0bprob> not found for neutral PFOs for group <D0bprob> in inputs. Setting to None
Feature <D0cprob> not found for neutral PFOs for group <D0cprob> in inputs. Setting to None
Feature <D0qprob> not found for neutral PFOs for group <D0qprob> in inputs. Setting to None
Feature <Z0bprob> not found for neutral PFOs for group <Z0bprob> in inputs. Setting to None
Feature <Z0cprob> not found for neutral PFOs for group <Z0cprob> in inputs. Setting to None
Feature <Z0qprob> not found for neutral PFOs for group <Z0qprob> in inputs. Setting to None
Feature <TrackUsedinPrimVertex> not found for neutral PFOs for group <TrackUsedinPrimVertex> in inputs. Setting to None
Feature <LeptonMomRelJet> not found for neutral PFOs for group <LeptonMomRelJet> in inputs. Setting to None
Feature <Kaoness> not found for neutral PFOs for group <Kaoness> in inputs. Setting to None
Feature <Chi2NDF> not found for ne

In [156]:
mappings_by_group

{'pf_points': {'source': [1, 2], 'target': [0, 1], 'target_none': []},
 'pf_features': {'source': [1,
   3,
   2,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34],
  'target': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32],
  'target_none': [33]},
 'sv_points': {'source': [1, 2], 'target': [0, 1], 'target_none': []},
 'sv_features': {'source': [1, 3, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
  'target': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
  'target_none': []}}

In [157]:
pf_charged_columns

{'JetIndex': 0,
 'RapidityRelJet': 1,
 'PhiRelJet': 2,
 'Rapidity': 3,
 'TransverseMomentumLOG': 4,
 'EnergyLOG': 5,
 'TransverseMomentumFracLOG': 6,
 'EnergyFracLOG': 7,
 'DeltaRJet': 8,
 'D0': 9,
 'D0Sig': 10,
 'Z0': 11,
 'Z0Sig': 12,
 '3DImpactPar': 13,
 '3DImpactParSig': 14,
 'Charge': 15,
 'IsElectron': 16,
 'IsMuon': 17,
 'IsChargedHadron': 18,
 'IsNeutralHadron': 19,
 'IsPhoton': 20,
 'D0bprob': 21,
 'D0cprob': 22,
 'D0qprob': 23,
 'Z0bprob': 24,
 'Z0cprob': 25,
 'Z0qprob': 26,
 'TrackUsedinPrimVertex': 27,
 'LeptonMomRelJet': 28,
 'Kaoness': 29,
 'Chi2NDF': 30,
 'LeptonMomFraction': 31,
 'WeightedKaonnessMomFrac': 32,
 'TrackJetDotProductNorm': 33,
 'HCalFrac': None}

In [158]:
mappings_by_group

{'pf_points': {'source': [1, 2], 'target': [0, 1], 'target_none': []},
 'pf_features': {'source': [1,
   3,
   2,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32,
   33,
   34],
  'target': [0,
   1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20,
   21,
   22,
   23,
   24,
   25,
   26,
   27,
   28,
   29,
   30,
   31,
   32],
  'target_none': [33]},
 'sv_points': {'source': [1, 2], 'target': [0, 1], 'target_none': []},
 'sv_features': {'source': [1, 3, 2, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
  'target': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
  'target_none': []}}

In [ ]:
from typing import Optional



In [176]:
calc_df_specification(10, parameters)

{'pf_mask': ('B', (10, 110)),
 'sv_mask': ('B', (10, 2)),
 'labels': ('B', 10),
 'pf_points': ('f', (10, 110, 2)),
 'pf_features': ('f', (10, 110, 34)),
 'sv_points': ('f', (10, 2, 2)),
 'sv_features': ('f', (10, 2, 14))}

In [177]:
h5p = '/afs/desy.de/group/flc/pool/bliewert/MarlinWorkdirs/ZHH/scripts/test.h5'
os.remove(h5p)
rb = ur.open('/afs/desy.de/group/flc/pool/bliewert/MarlinWorkdirs/ZHH/scripts/AIDAFile_.root.root')

a = root_files_to_hdf5(h5p, parameters, [rb], 0, lengths=[rb['Jets'].num_entries], file_fraction=1.0)

  0%|          | 0/1 [00:00<?, ?it/s]

Processed 0/54 jets...: 100%|██████████| 1/1 [00:00<00:00, 18.91it/s]                        


In [14]:
pf_charged_column_list = []

for key, value in pf_charged_columns.items():
    if value is not None:
        pf_charged_column_list.append(key)

In [15]:
a = ttree_to_array(rb['ChargedJetConstituents'], columns=pf_charged_column_list, force_type=np.float32)

In [21]:
from typing import Optional, cast
import torch.utils.data as torch_data
import os.path as osp
import os

class RootDataset(torch_data.Dataset):
    """Map-style dataset for using ROOT files. Converts inputs to HDF5
    for fast reading of individual events or a list of events.

    Args:
        hdf5_filepath (str): can be omitted only if root_files is provided
        root_files (list[str]): can be omitted only if hdf5_filepath is provided
    """
    
    def __init__(self, hdf5_filepath:str, root_files:Optional[list[str]]=None, label:Optional[int]=None, transform=None,
                 file_fraction:Optional[int]=1., device:Optional[torch.device]=None, verbose_fn:Callable=lambda x: x):
        
        if not osp.isfile(hdf5_filepath):
            if root_files is None or label is None:
                raise ValueError('hdf5_filepath not found and either root_files or label not provided')
            
            root_files = root_files
            root_handles = [ur.open(root_file) for root_file in root_files]
            file_lengths = [rh['Jets'].num_entries for rh in root_handles]
            
            self._length = int(np.floor(np.sum(file_lengths)*file_fraction))
            
            verbose_fn(f'HDF5 file not found, converting {len(self)} ROOT files with a total of {self._length} events...')
            
            root_files_to_hdf5(hdf5_filepath, parameters, root_handles, label, lengths=file_lengths, file_fraction=1.0)
        else:
            self._h5f = h5py.File(hdf5_filepath, 'r')
            self._length = len(self._h5f['labels'])
        
        self._transform = transform
        self._device = device
        
    def __len__(self):
        return self._length
    
    def __getitem__(self, index):
        sample = {}
        
        
        sample = {
            'pf_points': torch.tensor(self.pf_points[index], dtype=torch.float32, device=self.device),
            'pf_features': torch.tensor(self.pf_features[index], dtype=torch.float32, device=self.device),
            'pf_mask': torch.tensor(self.pf_mask[index], dtype=torch.bool, device=self.device),
            'sv_points': torch.tensor(self.sv_points[index], dtype=torch.float32, device=self.device),
            'sv_features': torch.tensor(self.sv_features[index], dtype=torch.float32, device=self.device),
            'sv_mask': torch.tensor(self.sv_mask[index], dtype=torch.bool, device=self.device),
            'labels': torch.tensor(self.labels[index], device=self.device).type(torch.LongTensor)
        }

        if self._transform:
            sample = self._transform(sample)

        return sample, sample['labels']
    
def _cleanup_nans(pf_features, pf_mask, sv_features, sv_mask):
    # Prepare dataset, calculate mean,stddev
    pf_features[(pf_features == -10.)] = np.nan
    pf_features[~pf_mask[:, :, 0]] = np.nan
    
    pf_mean = np.nanmean(pf_features, axis=(0,1))
    pf_std = np.nanstd(pf_features, axis=(0,1))

    sv_features[(sv_features == -10.)] = np.nan
    sv_features[~sv_mask] = np.nan
    
    sv_mean = np.nanmean(sv_features, axis=(0,1))
    sv_std = np.nanstd(sv_features, axis=(0,1))

    # rewrite NaNs
    pf_features[np.isnan(pf_features)] = 0
    sv_features[np.isnan(sv_features)] = 0